<a href="https://colab.research.google.com/github/victorsa2/sales_meli/blob/main/sales_meli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preparando o Ambiente no Github

In [2]:
!git clone https://github.com/victorsa2/sales_meli.git

Cloning into 'sales_meli'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 63 (delta 23), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (63/63), 281.09 KiB | 1.28 MiB/s, done.
Resolving deltas: 100% (23/23), done.


## Subindo o arquivo .env com as variáveis no drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from google.colab import files
uploaded = files.upload()

Saving .env to .env


##Instalando Dependencias não disponíveis no colab

In [5]:
pip install python-dotenv

In [6]:
import requests
import webbrowser
from dotenv import load_dotenv
import os

#Consultando API do Mercado Livre

##Obter Access Token e User ID

In [7]:
load_dotenv('/content/.env')

APP_ID = os.getenv("APP_ID")
APP_SECRET = os.getenv("APP_SECRET")
REDIRECT_URI = os.getenv("REDIRECT_URI")

def get_access_token_and_user_id():

    auth_url = f'https://auth.mercadolivre.com.br/authorization?response_type=code&client_id={APP_ID}&redirect_uri={REDIRECT_URI}'
    return auth_url

def request_token(authorization_code):
    token_url = 'https://api.mercadolibre.com/oauth/token'
    payload = {
        'grant_type': 'authorization_code',
        'client_id': os.getenv("APP_ID"),
        'client_secret': os.getenv("APP_SECRET"),
        'code': authorization_code,
        'redirect_uri': os.getenv("REDIRECT_URI")
    }

    response = requests.post(token_url, data=payload)
    token_data = response.json()

    access_token = token_data['access_token']
    user_id = token_data['user_id']

    return access_token, user_id

if __name__ == "__main__":
    auth_url = get_access_token_and_user_id()
    print("Abra o seguinte URL no seu navegador para obter o código de autorização:")
    print(auth_url)

    authorization_code = input("Insira o código de autorização: ")
    access_token, user_id = request_token(authorization_code)


Abra o seguinte URL no seu navegador para obter o código de autorização:
https://auth.mercadolivre.com.br/authorization?response_type=code&client_id=5500572046072825&redirect_uri=https://victorsa.netlify.app
Insira o código de autorização: TG-66fbf733ed58330001ef1cb3-478295390


##Obter dimensão de Categorias

In [8]:
import requests
import pandas as pd

def fetch_categories():
    url = "https://api.mercadolibre.com/sites/MLB/categories"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erro ao obter categorias: {response.status_code}")
        return []

categories_data = fetch_categories()
categories_list = []

for category in categories_data:
    categories_list.append({
        'id_category': category['id'],
        'name': category['name'],
    })

dim_Category = pd.DataFrame(categories_list)
dim_Category.to_csv('category.csv',encoding='utf-8-sig')

print("Dimensão Categorias:")
print(dim_Category.head())

Dimensão Categorias:
  id_category                      name
0     MLB5672  Acessórios para Veículos
1   MLB271599                      Agro
2     MLB1403       Alimentos e Bebidas
3     MLB1071                   Animais
4     MLB1367   Antiguidades e Coleções


## Consumindo API por produtos para conseguir alguns vendedores de exemplo, pesquisando por todas as categorias do Mercado Livre.


In [9]:
def fetch_categories():
    url = "https://api.mercadolibre.com/sites/MLB/categories"
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erro ao obter categorias: {e}")
        return []

def fetch_sales_in_category(category_id, access_token, site_id):
    url = f'https://api.mercadolibre.com/sites/{site_id}/search?category={category_id}'
    headers = {'Authorization': f'Bearer {access_token}'}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()['results']
    except requests.exceptions.RequestException as e:
        print(f"Erro ao obter vendas na categoria {category_id}: {e}")
        return []

ACCESS_TOKEN = access_token
SITE_ID = 'MLB'

sales_list = []
categories_data = fetch_categories()

for category in categories_data:
    category_id = category['id']
    print(f"\nBuscando vendas na categoria: {category['name']} (ID: {category_id})")

    sales_data = fetch_sales_in_category(category_id, ACCESS_TOKEN, SITE_ID)

    if sales_data:
        for item in sales_data:
            if 'seller' in item and 'nickname' in item['seller']:
                sales_list.append({
                    'seller_id': item['seller']['id'],
                    'seller_nickname': item['seller']['nickname'],
                    'category_id': category['id'],
                    'category': category['name'],
                    'item_id': item['id'],
                    'item_name': item['title'],
                    'currency': item['currency_id'],
                    'price': item['price']
                })
    else:
        print("Nenhum resultado encontrado para esta categoria.")

df_sales = pd.DataFrame(sales_list)
df_sales.to_csv('products.csv',encoding='utf-8-sig')

print(df_sales)


Buscando vendas na categoria: Acessórios para Veículos (ID: MLB5672)

Buscando vendas na categoria: Agro (ID: MLB271599)

Buscando vendas na categoria: Alimentos e Bebidas (ID: MLB1403)

Buscando vendas na categoria: Animais (ID: MLB1071)

Buscando vendas na categoria: Antiguidades e Coleções (ID: MLB1367)

Buscando vendas na categoria: Arte, Papelaria e Armarinho (ID: MLB1368)

Buscando vendas na categoria: Bebês (ID: MLB1384)

Buscando vendas na categoria: Beleza e Cuidado Pessoal (ID: MLB1246)

Buscando vendas na categoria: Brinquedos e Hobbies (ID: MLB1132)

Buscando vendas na categoria: Calçados, Roupas e Bolsas (ID: MLB1430)

Buscando vendas na categoria: Câmeras e Acessórios (ID: MLB1039)

Buscando vendas na categoria: Carros, Motos e Outros (ID: MLB1743)

Buscando vendas na categoria: Casa, Móveis e Decoração (ID: MLB1574)

Buscando vendas na categoria: Celulares e Telefones (ID: MLB1051)

Buscando vendas na categoria: Construção (ID: MLB1500)

Buscando vendas na categoria: El

# Exploração e Tratamento

Adicionando o arquivo gerado pelo GPT no ambiente do colab

In [30]:
import pandas as pd

path_category = '/content/category.csv'
path_products = '/content/products.csv'

df_category = pd.read_csv(path_category, sep=',')
df_products = pd.read_csv(path_products, sep=',')

print('Informações dataframe Category:\n')
shape_category = df_category.info()

print('\nInformações dataframe Products:\n')
shape_products = df_products.info()

Informações dataframe Category:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   32 non-null     int64 
 1   id_category  32 non-null     object
 2   name         32 non-null     object
dtypes: int64(1), object(2)
memory usage: 896.0+ bytes

Informações dataframe Products:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1582 entries, 0 to 1581
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       1582 non-null   int64  
 1   seller_id        1582 non-null   int64  
 2   seller_nickname  1582 non-null   object 
 3   category_id      1582 non-null   object 
 4   category         1582 non-null   object 
 5   item_id          1582 non-null   object 
 6   item_name        1582 non-null   object 
 7   currency         1549 non-null   object 
 8   

In [33]:
category_null = df_category.isnull().sum() > 0
print('Verificando nulos no dataframe Category:\n')
print(category_null)

products_null = df_products.isnull().sum() > 0
print('Verificando nulos no dataframe Products:\n')
print(products_null)

Verificando nulos no dataframe Category:

Unnamed: 0     False
id_category    False
name           False
dtype: bool
Verificando nulos no dataframe Products:

Unnamed: 0         False
seller_id          False
seller_nickname    False
category_id        False
category           False
item_id            False
item_name          False
currency            True
price               True
dtype: bool


In [26]:
df_price_null = df_products[df_products['price'].isnull()]
print('Linhas com valores nulos na coluna price:\n')
print(df_price_null)

Linhas com valores nulos na coluna price:

      Unnamed: 0  seller_id            seller_nickname category_id  category  \
1489        1489  169213892            BRILHYSEMIJOIAS     MLB1540  Serviços   
1490        1490  209363479                 SUN HAPPY.     MLB1540  Serviços   
1491        1491   29094661                ALUGUEFLUKE     MLB1540  Serviços   
1492        1492   89551550            ZERICARDOGUEDES     MLB1540  Serviços   
1493        1493   42451372             RICARDOYOUSSEF     MLB1540  Serviços   
1494        1494  104637759                DARLAN 1434     MLB1540  Serviços   
1495        1495     519584                  GARIDA_ML     MLB1540  Serviços   
1496        1496   56338640              SAMPA_MVENDAS     MLB1540  Serviços   
1497        1497  177129516             FRETEPONTUALSP     MLB1540  Serviços   
1499        1499  443589211           COMAHOCOMMATHOSP     MLB1540  Serviços   
1500        1500   56338640              SAMPA_MVENDAS     MLB1540  Serviços 


Retirando de todas as tabelas a categoria de "Serviços"

In [13]:
df_products = df_products[df_products['category'] != 'Serviços']
df_category = df_category[df_category['name'] != 'Serviços']

products_null = df_products.isnull().sum() > 0
print('Verificando nulos no dataframe Products:\n')
print(products_null)

Verificando nulos no dataframe Products:

Unnamed: 0         False
seller_id          False
seller_nickname    False
category_id        False
category           False
item_id            False
item_name          False
currency           False
price              False
dtype: bool


In [34]:
df_products.to_csv('data_processed/products.csv',encoding='utf-8-sig')
df_category.to_csv('data_processed/category.csv',encoding='utf-8-sig')